# Building Semantic Memory with Embeddings

So far, we've mostly been treating the kernel as a stateless orchestration engine.
We send text into a model API and receive text out. 

In a [previous notebook](04-kernel-arguments-chat.ipynb), we used `kernel arguments` to pass in additional
text into prompts to enrich them with more data. This allowed us to create a basic chat experience. 

However, if you solely relied on kernel arguments, you would quickly realize that eventually your prompt
would grow so large that you would run into a the model's token limit. What we need is a way to persist state
and build both short-term and long-term memory to empower even more intelligent applications. 

To do this, we dive into the key concept of `Semantic Memory` in the Semantic Kernel. 

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.5.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.5.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Sqlite, 1.5.0-alpha"
#r "nuget: System.Linq.Async, 6.0.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.5.0 Microsoft.SemanticKernel.Connectors.Sqlite, 1.5.0-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.5.0-alpha System.Linq.Async, 6.0.1

In order to use memory, we need to instantiate the Memory Plugin with a Memory Storage
and an Embedding backend. In this example, we make use of the `VolatileMemoryStore`
which can be thought of as a temporary in-memory storage (not to be confused with Semantic Memory).

This memory is not written to disk and is only available during the app session.

When developing your app you will have the option to plug in persistent storage
like Azure Cosmos Db, PostgreSQL, SQLite, etc. Semantic Memory allows also to index
external data sources, without duplicating all the information, more on that later.

In [2]:
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.Sqlite;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Memory functionality is experimental
#pragma warning disable SKEXP0003, SKEXP0011, SKEXP0052, SKEXP0028

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var index = !File.Exists("index.db");

var store = await SqliteMemoryStore.ConnectAsync("index.db");

var memoryBuilder = new MemoryBuilder();

if (useAzureOpenAI)
{
    memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
        "text-embedding-ada-002",
        azureEndpoint, 
        apiKey,
        "model-id");
}
else
{
    memoryBuilder.WithOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey);
}


memoryBuilder.WithMemoryStore(store);

var memory = memoryBuilder.Build();

Let's try searching the memory:

In [3]:
const string MemoryCollectionName = "CocktailCollection";

var questions = new[]
{
    "list of good sparkling cocktails",
};

foreach (var q in questions)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
    Console.WriteLine(q + " " + response?.Metadata.Text);
}

list of good sparkling cocktails Bellini, Glass: champagne-flute, Category: Sparkling Cocktail, Ingredients: 10 cl Prosecco, 5 cl Peach puree, Garnish: , Preparation: Pour peach puree into chilled glass and add sparkling wine. Stir gently. Variations: Puccini (fresh mandarin juice), Rossini (fresh strawberry puree), Tintoretto (fresh pomegranate juice)


In [4]:
using Microsoft.SemanticKernel.Plugins.Memory;

#pragma warning disable SKEXP0052

// TextMemoryPlugin provides the "recall" function
kernel.ImportPluginFromObject(new TextMemoryPlugin(memory));

Error: (6,35): error SKEXP0003: 'Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin' is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

In [5]:
#pragma warning disable SKEXP0003
var ingredient = await InteractiveKernel.GetInputAsync("Type an alcohol ingredient?");

IAsyncEnumerable<MemoryQueryResult> queryResults =
                memory.SearchAsync(MemoryCollectionName, ingredient, limit: 3, minRelevanceScore: 0.77);

In [6]:
#pragma warning disable SKEXP0003
// Keep a list of the memories
StringBuilder promptData = new StringBuilder();

await foreach (MemoryQueryResult r in queryResults)
{
    promptData.Append(r.Metadata.Text+"\n\n");
}

// Final augmented text
var augmentedText = promptData.ToString();
Console.WriteLine($"User:\n{ingredient}\n\nNearest results:\n{augmentedText}")

User:
vodka

Nearest results:
Black Russian, Glass: old-fashioned, Category: After Dinner Cocktail, Ingredients: 5 cl Vodka, 2 cl Coffee liqueur, Garnish: , Preparation: Build into old fashioned glass filled with ice cubes. Stir gently. Note: for White Russian, float fresh cream on the top and stir gently.

Vesper, Glass: martini, Category: Before Dinner Cocktail, Ingredients: 6 cl Gin, 1.5 cl Vodka, 0.75 cl Lillet Blonde, Garnish: Lemon twist, Preparation: Shake and strain into a chilled cocktail glass.

Cosmopolitan, Glass: martini, Category: All Day Cocktail, Ingredients: 4 cl Vodka (Citron Vodka), 1.5 cl Triple Sec (Cointreau), 1.5 cl Lime juice, 3 cl Cranberry juice, Garnish: Lime slice, Preparation: Shake with ice cubes. Strain into a large cocktail glass.




In [7]:

const string ragFunctionDefinition = @"
ChatBot can only have a conversation with relevant information below.
It can give explicit instructions or say 'Beats me, I'm just here for the drink' if it does not have an answer.

Relevant information:
{{$data}}

Chat:
{{$history}}
User: {{$userInput}}
ChatBot: ";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.8,
    TopP = 0.5
};


var ragFunction = kernel.CreateFunctionFromPrompt(ragFunctionDefinition, executionSettings);


In [8]:
#pragma warning disable SKEXP0052

var arguments = new KernelArguments();

var question = $"list spirit that includes {ingredient}"; // await InteractiveKernel.GetInputAsync("What is your query?");

arguments["userInput"] = question;
arguments["data"] = augmentedText;

var result = await kernel.InvokeAsync(ragFunction, arguments);

Console.WriteLine(question);
Console.WriteLine(result);



list spirit that includes vodka
Black Russian, Vesper, and Cosmopolitan are cocktails that include vodka as one of the main ingredients.
